In [ ]:
from utils import *

In [ ]:
generate_inventory(verbose=True)

In [ ]:
get_character_needs(['Melue', 'Zegna'])

In [ ]:
generate_auction_history(verbose=True)

In [ ]:
# Gets relevant list of items
with open(f'config/items.yaml', 'r') as f:
    items = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:
# Gets relevant list of items
with open(f'config/self_demand.yaml', 'r') as f:
    self_demand = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:
#self_demand

In [ ]:
for consumable, details in items.items():
    if consumable in self_demand:
        items[consumable]['self_demand'] = self_demand[consumable]
    else:
        pass #print(consumable)

In [ ]:
with open('config/recipes.yaml', 'r') as f:
    recipes = yaml.load(f, Loader=yaml.FullLoader) 
    
flat_recipes = {}
for trade_good, recipe in recipes.items():
    flat_recipe = defaultdict(int)
    for ingredient, count in recipe.items():
        if ingredient in recipes:
            sub_recipe = recipes.get(ingredient, {})
            for sub_ingredient, sub_count in sub_recipe.items():
                flat_recipe[sub_ingredient] = count*sub_count
        else:
            flat_recipe[ingredient] = count
    flat_recipes[trade_good] = flat_recipe    

In [ ]:
for consume, details in items.items():
    if consume in flat_recipes:
        items[consume]['made_from'] = dict(flat_recipes[consume])

In [ ]:
# Gets relevant list of items
with open(f'config/items.yaml', 'w') as f:
    yaml.dump(items, f)

In [ ]:
df = pd.read_parquet('intermediate/auctions.parquet')

In [ ]:
sold = df[(df['categories']=='Sell')&(df['auction_type']=='sold')]

In [ ]:
price_mean, price_std, price_count = analyse_price(sold[sold['timestamp']>='2020-03-01'])

In [ ]:
price_count

In [ ]:
sold = df[(df['categories']=='Sell')&(df['auction_type']=='sold')]

In [ ]:
df = pd.read_parquet('intermediate/auctions.parquet')

recent_price, _, recent_count = analyse_price(df[df['timestamp']>='2020-03-01'])
historic_price, historic_std, historic_count = analyse_price(df)

In [ ]:
categories, _ = get_interesting_items_and_stacks()

In [ ]:
df

In [ ]:
categories

In [ ]:
df

In [ ]:
df[df['auction_type']=='sold'].groupby('item').sum()

In [ ]:
df[df['auction_type']=='failed'].groupby('item').sum()

In [ ]:
bought_items = list(categories['Snatch'].keys()) + list(categories['Buy'].keys())

In [ ]:
bought_price = recent_price.loc[[x for x in bought_items if x in recent_price.index], 'bought'].dropna().to_dict()

In [ ]:
# bought_price['Wild Steelbloom'] = '7000'

In [ ]:
sold_items = list(categories['Sell'].keys())

In [ ]:
sold_price = recent_price.loc[[x for x in sold_items if x in recent_price.index], ['sold']].dropna()

In [ ]:
sold_price['mat_cost'] = 0

In [ ]:
flat_recipes

In [ ]:
for potion in sold_price.index:
    recipe = recipes.get(potion, {})
    
    for ingredient, count in recipe.items():
        if ingredient in bought_price:
            sold_price.loc[potion, 'mat_cost'] += count * bought_price[ingredient]
        else:
            sub_recipe = recipes.get(ingredient, {})
            
            for sub_ingredient, sub_count in sub_recipe.items():
                if sub_ingredient in bought_price:
                    sold_price.loc[potion, 'mat_cost'] += count * sub_count * bought_price[sub_ingredient]

In [ ]:
potion = "Greater Shadow Protection Potion"

In [ ]:
flat_recipes

In [ ]:
bought_price['Wild Steelbloom'] = 7000
bought_price['Grave Moss'] = 4500
bought_price['Fadeleaf'] = 5000
bought_price['Winter Squid'] = 7000
bought_price['Tender Wolf Meat'] = 900
bought_price['Swiftthistle'] = 5400
bought_price['Briarthorn'] = 800
bought_price["Khadgar's Whiskers"] = 200

bought_price['Empty Vial'] = 3
bought_price['Leaded Vial'] = 32
bought_price['Crystal Vial'] = 400

In [ ]:
mat_cost = defaultdict(int)
for trade_good, recipe in flat_recipes.items():
    for ingredient, cost in recipe.items():
        mat_cost[trade_good] += bought_price[ingredient] * cost

In [ ]:
sold_price['mat_cost'] = 0

In [ ]:
for ind, row in sold_price.iterrows():
    if ind in mat_cost:
        sold_price.loc[ind, 'mat_cost'] = mat_cost[ind]

In [ ]:
sold_price

In [ ]:
sold_price['profit'] = sold_price['sold'] - sold_price['mat_cost']

In [ ]:
sold_price.sort_values('profit')